In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import ast

In [2]:
standard = ['True', 'Mostly True ','Half True ', 'Mostly False', 'False', 'Pants on Fire']
score = np.array([10, 8, 6, 4, 2, 0])

In [3]:
poli_df = pd.read_csv('politifact_data.csv')

In [4]:
poli_df.head()

,media,when/where,content,label,speaker,documented_time,percentages,check_nums,summaries,article
0,Instagram posts,"stated on October 28, 2023 in a screenshot sha...",“Haaretz investigation reveals discrepancies i...,false,Madison Czopek,"October 31, 2023",['0%' '0%' '2%' '7%' '67%' '21%'],[ 5 3 16 54 473 152],"['Haaretz, an Israeli newspaper, said on X tha...",A viral Oct. 28 social media post claimed that...
1,Scott Walker,"stated on May 30, 2023 in Interview:",“Wisconsin has historically … and I think larg...,barely-true,Laura Schulte,"October 31, 2023",['12%' '21%' '18%' '19%' '21%' '5%'],[26 45 39 41 44 11],['Although Wisconsin has voted for more Democr...,"In 2016, Wisconsin helped to swing the preside..."
2,Instagram posts,"stated on October 27, 2023 in a post:","“The airport in Salzburg, Austria, has a count...",false,Ciara O'Rourke,"October 30, 2023",['0%' '0%' '2%' '7%' '67%' '21%'],[ 5 3 16 54 473 152],[],A social media post poised to encourage people...
3,Viral image,"stated on October 27, 2023 in an Instagram post:",Video shows Palestinians pretending to be corp...,false,Ciara O'Rourke,"October 30, 2023",['0%' '1%' '2%' '4%' '62%' '28%'],[ 4 13 35 53 745 336],['This video is 10 years old and shows student...,The Gaza Health Ministry has said the Palestin...
4,Facebook posts,"stated on September 25, 2023 in a Facebook post:",The life span of a wind tower generator lasts ...,false,Loreben Tuquero,"October 30, 2023",['0%' '1%' '4%' '9%' '59%' '23%'],[ 24 50 108 247 1519 594],['A study by energy industry experts showed th...,Let’s clear the air. Do wind turbine component...


In [5]:
def transform_percentages(col):
    return sum(np.array(ast.literal_eval(col.replace('%', '').replace('\'', '').replace(' ', ',')))/100 * score)

In [6]:
# Propose credibility score/Reliable source

In [7]:
# from transformers import pipeline
# summarizer = pipeline("summarization", model="Falconsai/text_summarization")

In [8]:
# def summarization(col):
#     return (summarizer(col, max_length=150, min_length=30, do_sample=False))

In [9]:
# poli_df['summarized_points'] = poli_df['article'].apply(summarization)
# poli_df.to_csv('tmp_politifact.csv')

In [10]:
# summary = np.array(poli_df['summaries'])
# for i in range(summary):
#     if summary[i] == '[]':
#         summary[i] = '[' + summary_col[i][0]['summary_text'] + ']'
# temp_polidf['summaries'] = summary

In [11]:
statement_labels = {
    'pants-fire': 0,
    'false': 1,
    'barely-true': 2,
    'half-true': 3,
    'mostly-true': 4,
    'true': 5,
    'half-flip': 6,
    'no-flip': 7,
    'full-flop': 8
}

In [12]:
poli_df['credibility_score'] = poli_df['percentages'].apply(transform_percentages)
poli_df['label'] = poli_df['label'].map(statement_labels)

In [13]:
poli_df['content'] = poli_df['content'].str.replace("“", "").str.replace("”","").str.replace("\"", '')

In [14]:
cleaned_df = poli_df[['media', 'content', 'summaries', 'credibility_score', 'label']].reset_index()
cleaned_df

,index,media,content,summaries,credibility_score,label
0,0,Instagram posts,Haaretz investigation reveals discrepancies in...,"['Haaretz, an Israeli newspaper, said on X tha...",1.74,1
1,1,Scott Walker,Wisconsin has historically … and I think large...,['Although Wisconsin has voted for more Democr...,5.14,2
2,2,Instagram posts,"The airport in Salzburg, Austria, has a counte...",[],1.74,1
3,3,Viral image,Video shows Palestinians pretending to be corp...,['This video is 10 years old and shows student...,1.60,1
4,4,Facebook posts,The life span of a wind tower generator lasts ...,['A study by energy industry experts showed th...,1.86,1
...,...,...,...,...,...,...
25610,25610,Mike Huckabee,I'll tell you what I can tell this country: If...,[],5.12,1
25611,25611,Rudy Giuliani,Democrats are kind of falling over each other ...,[],4.70,3
25612,25612,Bill Richardson,"I used tax cuts to help create over 80,000 job...",[],6.30,4
25613,25613,Hillary Clinton,"Corporate profits are up, CEO pay is up, but a...",[],6.44,5


In [35]:
X = cleaned_df[['content', 'credibility_score']]
y = cleaned_df['label']

In [41]:
filtered_df = cleaned_df[cleaned_df['label'].isin(range(6))]
min_count = filtered_df['label'].value_counts().min()
sampled_df = filtered_df.groupby('label').apply(lambda group: group.sample(min_count)).reset_index(drop=True).drop(columns = 'index')
sampled_df

,media,content,summaries,credibility_score,label
0,Viral image,A photograph shows Barack Obama bowing to Ayat...,[],1.60,0
1,Donald Trump,[U.S. Rep. Conor Lamb] said he would NOT vote ...,"['On Memorial Day, President Donald Trump atta...",3.16,0
2,Mitt Romney,We're only inches away from no longer being a ...,[],5.16,0
3,Viral image,"Says Ronald Reagan said, Nancy Pelosi is extre...",[],1.60,0
4,Facebook posts,Biden concedes 6 million votes.,['Biden misspoke about the the number of voter...,1.86,0
...,...,...,...,...,...
17461,Mitt Romney,Half the kids coming out of college this year ...,[],5.16,5
17462,Jenny Wilson,Says Mitt Romney is using a million of his pre...,[],10.00,5
17463,Chris Abele,90 percent of Americans support universal back...,[],4.70,5
17464,Heather McGhee,The Republican presidential nominee hasn’t won...,[],6.60,5


In [44]:
X = sampled_df[['content','credibility_score']]
y = sampled_df['label']

In [29]:
X_preprocessed = X.copy()

tfidf_vectorizer = TfidfVectorizer(norm = "l2")
X_summaries = tfidf_vectorizer.fit_transform(X_preprocessed['content'])

X_summaries_reset = pd.DataFrame(X_summaries.toarray()).reset_index(drop=True)
X_preprocessed_reset = X_preprocessed['credibility_score'].reset_index(drop=True)
X_features = pd.concat([X_summaries_reset, X_preprocessed_reset], axis=1)

In [45]:
y = y.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

In [46]:
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.454779622209502


In [ ]:
# Limitation: Too much repetitive except 1, can apply the same approach but use it on a more realistic dataset
# Need to come up with ways to summarize the article

In [ ]:
# TODO: just use the summariztion model to summarize the article at this point
# And then do it again